In [34]:
import json, pycm, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from datetime import datetime
from scipy.stats import spearmanr
from IPython.display import display, Markdown

In [35]:
datapoints = json.load(open('results_trial-v1_usp_v5_gpt4-turbo_stage_2.json', 'r'))
datapoints = [ dp for dp in datapoints if "predicted" in dp ]
results = pd.DataFrame.from_records(datapoints)
cm = pycm.ConfusionMatrix(results["label"].tolist(), results["predicted"].tolist(), digit=2, classes=[ 'Hallucination', 'Not Hallucination' ])

## Performance: N = 80, accuracy = 0.900, time elapsed = 0:05:38

In [36]:
start = datetime.fromisoformat(results["timestamp"].min())
end = datetime.fromisoformat(results["timestamp"].max())
elapsed = end - start
display(Markdown(f'## Performance: N = {cm.POP["Hallucination"]}, accuracy = {cm.ACC["Hallucination"]:.3f}, time elapsed = {elapsed}'))
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
df_cm = pd.DataFrame(cm.matrix).T.fillna(0)
sns.heatmap(df_cm, annot=True, fmt='d', cmap="YlGnBu", ax=axes[0])
axes[0].set_title("SHROOM Trial v1 - USP")
axes[0].set(xlabel='Predicted', ylabel='Actual')
sns.regplot(x="p(Hallucination)", y="predicted_p", data=results, scatter_kws={"s": 20}, line_kws={"color": "red", "alpha": 0.7, "lw": 2}, lowess=True, ax=axes[1])
rho, pval = spearmanr(results['p(Hallucination)'], results['predicted_p'])
axes[1].set_title(f'Spearman’s ρ = {rho:.3f} (p={pval:.2f})')
axes[1].set_xlabel('p(Hallucination)')
axes[1].set_ylabel('Estimated p(Hallucination)')
plt.tight_layout()
plt.show()


In [37]:
def display_errors(results):
    fns = results[((results["label"] == 'Hallucination') & (results["predicted"] == 'Not Hallucination'))].copy()
    fns["src"] = fns["src"].str.replace('<','&lt;')
    fns["src"] = fns["src"].str.replace('>','&gt;')
    fns = fns.drop("model", axis=1)
    # fns = fns.drop("label", axis=1)
    fns = fns.drop("labels", axis=1)
    # fns = fns.drop("predicted", axis=1)
    # fns = fns.drop("demos", axis=1)
    # fns = fns.drop("predictions", axis=1)
    # fns = fns.drop("timestamp", axis=1)
    if len(fns) > 0:
        fns_styler = fns.style.set_properties(**{"text-align": "left", "vertical-align" : "top", "overflow-wrap": "break-word"}).hide(axis="index")
        display(Markdown(f'## False negatives'))
        display(fns_styler)
    fps = results[((results["label"] == 'Not Hallucination') & (results["predicted"] == 'Hallucination'))].copy()
    fps["src"] = fps["src"].str.replace('<','&lt;')
    fps["src"] = fps["src"].str.replace('>','&gt;')
    fps = fps.drop("model", axis=1)
    # fps = fps.drop("label", axis=1)
    fps = fps.drop("labels", axis=1)
    # fps = fps.drop("predicted", axis=1)
    # fps = fps.drop("demos", axis=1)
    # fps = fps.drop("predictions", axis=1)
    # fps = fps.drop("timestamp", axis=1)
    if len(fps) > 0:
        fps_styler = fps.style.set_properties(**{"text-align": "left", "vertical-align" : "top", "overflow-wrap": "break-word"}).hide(axis="index")
        display(Markdown(f'## False positives'))
        display(fps_styler)

display_errors(results)

## False positives

hyp,ref,src,tgt,task,label,p(Hallucination),predicted,predicted_p,timestamp,rationale
(nautical) A halyard.,tgt,"Haul away , keeping strain on both parts of the halyard so that the <define> pigstick </define> remains vertical as it goes up and does n't foul the spreaders .",(nautical) A staff that carries a flag or pennant above the mast of a sailboat.,DM,Not Hallucination,0.333333,Hallucination,1.000000,2023-11-11T05:12:55Z,"The output is a hallucination. The target defines ""pigstick"" as a staff that carries a flag or pennant above the mast of a sailboat, which is a specific piece of equipment related to the display of flags. The output incorrectly defines ""pigstick"" as a halyard, which is a rope or line used for hoisting sails, flags, or yards. While a pigstick may be associated with the use of a halyard because it is raised above the mast, it is not itself a halyard. The output provides information that is not supported by the reference, thus it is a hallucination."
"(Australia, New Zealand, colloquial) A hatfish.",tgt,"Noah did n't realize when he stayed on the job - he was then about 600 years old and built the ark and drove the animals in two by two , the <define> hatfish </define> and the kangaroo — — he did n't realize he was laying the foundation for an occupation for old men in New York , several years later .",(colloquial) Synonym of halibut,DM,Not Hallucination,0.333333,Hallucination,1.000000,2023-11-11T05:13:19Z,"The output is a hallucination. The target defines ""hatfish"" as a colloquial synonym for halibut, which is a type of flatfish known for its size and taste. The output, however, does not provide a definition at all; it merely repeats the term ""hatfish"" with a regional tag (Australia, New Zealand, colloquial), which does not constitute a definition. Since the output fails to define the term and instead just restates it with a regional label, it does not provide the necessary information and is therefore considered a hallucination."
(transitive) To cook (something) in a frying pan.,tgt,Innawong et al . ( 2006 ) conducted experiments to study the effect of using nitrogen gas on the quality of <define> fried </define> chicken nuggets during pressure frying in terms of moisture retention and reduction in fat [ … ],Cooked in a deep fryer or pressure fryer or the like after being coated (breaded) in batter; compare deep-fried.,DM,Not Hallucination,0.000000,Hallucination,1.000000,2023-11-11T05:14:57Z,"The output is a hallucination. The target specifically defines ""fried"" in the context of chicken nuggets as being cooked in a deep fryer or pressure fryer after being coated in batter, which is a specific cooking method that involves submerging food in hot oil or fat. The output, however, provides a more general definition of ""fried"" as to cook something in a frying pan, which does not necessarily involve the same method as deep frying or pressure frying. The output omits the specific context of deep frying or pressure frying and the coating process, which is essential to the target definition, thus providing information that is not supported by the reference."
That’s not happening at the moment.,either,That’s not happening.,That isn’t gonna happen.,PG,Not Hallucination,0.333333,Hallucination,1.000000,2023-11-11T05:15:34Z,"The output ""That’s not happening at the moment."" is a hallucination. The addition of ""at the moment"" introduces a temporal limitation that is not present in the input ""That’s not happening."" or the target ""That isn’t gonna happen."" The original statements suggest a more definitive or general negation of an event occurring, while the output implies that the event is not occurring currently but could potentially occur later. This added temporal context is unsupported by the reference input and target, thus making the output a hallucination."
Do you love her?,either,Est-ce que tu l'aimes ?,Do you love him?,MT,Not Hallucination,0.333333,Hallucination,1.000000,2023-11-11T05:16:12Z,"Th